In [ ]:
from instagrapi import Client
from instagrapi.exceptions import LoginRequired
import logging
import time
import sqlite3
import pandas as pd
from time import localtime
# import httperror
from urllib.error import HTTPError
import re

In [ ]:

# USERNAME = 'coolpolyester'
# PASSWORD = 'Aaronjk12'

In [ ]:
# Should not need to run this ever again. 
# cl = Client()
# cl.login(USERNAME, PASSWORD)
# cl.delay_range = [3, 10]
# cl.dump_settings("session.json")

In [ ]:
logger = logging.getLogger()


def login_user(cl):
    """
    Attempts to login to Instagram using either the provided session information
    or the provided username and password.
    """
    USERNAME = 'coolpolyester'
    PASSWORD = 'Aaronjk12'
    
    session = cl.load_settings("session.json")

    login_via_session = False
    login_via_pw = False

    if session:
        try:
            cl.set_settings(session)
            cl.login(USERNAME, PASSWORD)

            # check if session is valid
            try:
                cl.get_timeline_feed()
            except LoginRequired:
                logger.info(
                    "Session is invalid, need to login via username and password"
                )

                old_session = cl.get_settings()

                # use the same device uuids across logins
                cl.set_settings({})
                cl.set_uuids(old_session["uuids"])

                cl.login(USERNAME, PASSWORD)
            login_via_session = True
        except Exception as e:
            logger.info("Couldn't login user using session information: %s" % e)

    if not login_via_session:
        try:
            logger.info(
                "Attempting to login via username and password. username: %s" % USERNAME
            )
            if cl.login(USERNAME, PASSWORD):
                login_via_pw = True
        except Exception as e:
            logger.info("Couldn't login user using username and password: %s" % e)

    if not login_via_pw and not login_via_session:
        raise Exception("Couldn't login user with either password or session")

In [ ]:
cl = Client()
login_user(cl)
# user_id = cl.user_id_from_username('actuarialclubuw')
# time.sleep(3)

In [ ]:
conn = sqlite3.connect('clubs.db')
c = conn.cursor()
# instaUrls = c.execute(r'SELECT club_id, instagramUrl FROM clubs WHERE instagramUrl;').fetchall()

In [ ]:
clubs_df = pd.read_sql_query("SELECT club_id, instagramUrl, insta_id FROM clubs;", conn)
# strftime('%Y-%m-%d %H:%M:%S', localtime(1347517370))
# cl.delay_range = [3, 10]

def extract_first_resource(url):
    # This regular expression will find the first resource after .com/ in the URL
    match = re.search(r'\.com/([^/?]+)', url)
    return match.group(1) if match else None


for index, club in clubs_df.iterrows():
    try:
        if club["instagramUrl"] != "" and club["insta_id"] is None:
            url = club["instagramUrl"]
            username = extract_first_resource(url)
            print(f'Requesting {username}\'s user_id')
            user_id = cl.user_id_from_username(username)

            # Put user_id into database
            c.execute('UPDATE clubs SET insta_id = ? WHERE club_id = ?;', (user_id, club['club_id']))
            conn.commit()

            formatted_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
            print(index, f'{club["instagramUrl"]}\'s user_id is {user_id} at {formatted_time}')
            
        elif club["instagramUrl"] != "":
            print(index, club["instagramUrl"] + "'s user_id is already in database")
    except HTTPError as e:
        print(e)
        continue

In [ ]:
# Get media

user_ids = pd.read_sql_query("SELECT insta_id FROM clubs WHERE insta_id IS NOT NULL;", conn)
user_ids.iloc[0].values[0]

In [ ]:
insta_ids = pd.read_sql_query("SELECT insta_id FROM clubs WHERE insta_id IS NOT NULL;", conn)
clubs = pd.read_sql_query("SELECT * FROM clubs;", conn)
seen_media = set(pd.read_sql_query("SELECT media_id FROM media;", conn)['media_id'])

for index, insta_id in insta_ids.iterrows():
    print(index)
    try:    
        media = cl.user_medias(insta_id, amount=10)
        for post in media:
            # Check if post is already in database
            if int(post.pk) in seen_media:
                pass
            else:
                # Insert post into database
                media_id = post.pk
                # club_id = clubs[clubs['insta_id'] == insta_id].iloc[0]['club_id']
                club_id = clubs.loc[clubs['insta_id'] == insta_id.values[0]]['club_id'].values[0]
                club_name = clubs.loc[clubs['insta_id'] == insta_id.values[0]]['club_name'].values[0]
                post_date = post.taken_at
                if post.resources:
                    first_image_url = post.resources[0].thumbnail_url
                else:
                    first_image_url = post.thumbnail_url
                media_src = 'Instagram'
                c.execute('INSERT INTO media VALUES (?, ?, ?, ?, ?, ?);', (media_id, club_id, first_image_url, media_src, post_date, club_name))
                seen_media.add(media_id)
                conn.commit()
                
    except Exception as e:
        print(e)
        continue


In [ ]:
insta_ids = pd.read_sql_query("SELECT insta_id FROM clubs WHERE insta_id IS NOT NULL;", conn)
clubs = pd.read_sql_query("SELECT * FROM clubs;", conn)
seen_media = set(pd.read_sql_query("SELECT media_id FROM media2;", conn)['media_id'])

for index, insta_id in insta_ids.iterrows():
    print(index)
    try:    
        media = cl.user_medias(insta_id, amount=10)
        for post in media:
            # Check if post is already in database
            if int(post.pk) in seen_media:
                pass
            else:
                # Insert post into database
                media_id = post.pk
                # club_id = clubs[clubs['insta_id'] == insta_id].iloc[0]['club_id']
                club_id = clubs.loc[clubs['insta_id'] == insta_id.values[0]]['club_id'].values[0]
                club_name = clubs.loc[clubs['insta_id'] == insta_id.values[0]]['club_name'].values[0]
                post_date = post.taken_at
                if post.resources:
                    first_image_url = post.resources[0].thumbnail_url
                else:
                    first_image_url = post.thumbnail_url
                media_src = 'Instagram'
                media_caption = post.caption_text
                like_count = post.like_count
                c.execute('INSERT INTO media2 VALUES (?, ?, ?, ?, ?, ?, ?, ?);', (media_id, club_id, first_image_url, media_src, post_date, club_name, media_caption, like_count))
                seen_media.add(media_id)
                conn.commit()
                
    except Exception as e:
        print(e)
        continue